In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [10]:
keras = tf.keras
imgX = 720
imgY = 720
EPOCHS = 10

In [11]:
classes = os.listdir('./assets')
classes = classes[:2]
classes

['daisy', 'dandelion']

In [4]:
def read_img(path):
    img = cv2.imread(path, 0)
    img.resize(imgX,imgY)
    print(img.shape)
    return img

In [ ]:
# get training data from all classes into one
images = []
labels = []
# for i, c in enumerate(classes):
#     for f in os.listdir(f'./assets/{c}'):
#         img = read_img(f'./assets/{c}/{f}')
#         images.append(img)
#         labels.append(i)
print(set(labels))

In [ ]:
print(f'Total images: {len(images)}\nTotal labels: {len(labels)}\nImage Size: {images[0].shape}')

In [ ]:
# split in train and test
X_train, X_test = images[:int(len(images)*0.8)], images[int(len(images)*0.8):]
y_train, y_test = labels[:int(len(labels)*0.8)], labels[int(len(labels)*0.8):]
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# model = keras.Sequential()
# model.add(keras.layers.Flatten(input_shape=(imgX,imgY)))
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dense(128, activation='relu'))
# model.add(keras.layers.Dense(len(set(labels)), activation='softmax'))
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train,epochs=EPOCHS)
# model.save('flower_model.h5')

In [6]:
model = keras.models.load_model('flower_model.h5')
# loss, accuracy = model.evaluate(X_test, y_test)
# loss, accuracy

In [7]:
def predict_image(path):
    img = read_img(path)
    img = np.array([img])
    x = model.predict(img)
    return classes[np.argmax(x[0])]

In [13]:
predict_image('./assets/daisy/10994032453_ac7f8d9e2e.jpg')

(720, 720)
1/1 [==============================] - 0s 54ms/step


'daisy'